# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


#import gmaps
#import pandas as pd

# Google developer API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Input File (CSV)
input_data_file = "output_data/cities.csv"

city_data = pd.read_csv(input_data_file)
city_data.dropna(inplace = True) 
print(f"Number of records in the input data file: {len(city_data)}")
city_data.head()

Number of records in the input data file: 557


,city,country,datetime,lat,long,maxtemp,humidity,cloudiness,windspeed
0,Punta Arenas,CL,1633488223,-53.1500,-70.9167,42.91,61,0,12.66
1,Ushuaia,AR,1633488224,-54.8000,-68.3000,38.86,60,0,6.91
2,Bluff,NZ,1633488225,-46.6000,168.3333,65.80,80,100,7.94
3,Santa Maria,BR,1633488227,-29.6842,-53.8069,58.30,77,0,8.05
4,Mutare,ZW,1633488228,-18.9707,32.6709,60.37,95,55,3.69


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations

coordinates = city_data[["lat", "long"]]
#coordinates = [(-22,-42),(60,92),(-14,36)]



humidity = city_data["humidity"].astype(float)

#humidity = [(97),(98)]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_cities = city_data.loc[(city_data["maxtemp"]> 70) & 
                             (city_data["maxtemp"]< 80) & 
                             (city_data["windspeed"]< 10) &
                             (city_data["cloudiness"] == 0)]
#ideal_cities.dropna(inplace = True)
print(f"Number of cities meeting criteria: {len(ideal_cities)}")
ideal_cities

Number of cities meeting criteria: 11


,city,country,datetime,lat,long,maxtemp,humidity,cloudiness,windspeed
56,Umluj,SA,1633488302,25.0213,37.2685,78.60,56,0,9.48
64,San Patricio,US,1633488322,28.0170,-97.5169,79.84,49,0,5.44
70,Toowoomba,AU,1633488330,-27.5500,151.9667,75.94,17,0,6.11
247,Ubari,LY,1633488606,26.5921,12.7805,70.90,32,0,7.78
248,Vao,NC,1633488608,-22.6667,167.4833,73.65,82,0,9.66
332,Houma,US,1633488728,29.5958,-90.7195,75.20,90,0,5.12
377,Kiryat Gat,IL,1633488793,31.6100,34.7642,70.48,75,0,3.15
485,Quelimane,MZ,1633488907,-17.8786,36.8883,73.35,94,0,1.99
515,Estelle,US,1633489002,29.8458,-90.1067,75.99,78,0,4.29
524,Sydney,AU,1633488879,-33.8679,151.2073,78.62,35,0,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
